In [12]:
import numpy as np
def v(x, y, z=0):
    return np.array([x, y, z])

import tempfile
from pathlib import Path

from solid import *
from solid.utils import *
import viewscad

from subprocess import run


R=viewscad.Renderer()

wall_size=2
fanZ=15
fanX=58
fanY=55
fan=v(fanX, fanY, fanZ)

bedZ=17
bedX=50
bedY=100
bed=v(bedX, bedY, bedZ)

def XYZ(X=0, Y=0, Z=0):
    return lambda *args : translate(v(X, Y, Z))(*args)

def NULLOBJ():
    return cube(0.1)

keep_scad = False

def render_to_stl(name, object, stldir, keep_scad):
    if not keep_scad:
        tmp_dir=tempfile.TemporaryDirectory()
        tmp_dirpath=Path(tmp_dir.name)
        scad_dir=tmp_dirpath
        print(f'Not keeping scad files in {scad_dir.as_posix()}')
    else:
        scad_dir=Path("scad")
        scad_dir.mkdir(parents=True, exist_ok=True)
        tmp_dir=scad_dir
        tmp_dirpath=tmp_dir
        print(f'Keeping scad files in {scad_dir.as_posix()}')
    stldir.mkdir(parents=True, exist_ok=True)
    stlname = stldir / Path(name+".stl")
    scadname = scad_dir / Path(name+".scad")
    print(f'stlname == {stlname.as_posix()}')
    print(f'scadname == {scadname.as_posix()}')
    # generate valid openscad code and store it in file
    scad_render_to_file(object, scadname.as_posix())

    # run openscad and export to stl
    run(["openscad", "--render", "-o",  stlname.as_posix(), scadname.as_posix()])
    if not keep_scad:
        tmp_dir.cleanup()

def make_part(name, object, stldir=Path("stl")):
    render_to_stl(name, object, stldir, keep_scad)
    return object



In [4]:
@bom_part("Clamp Jaw")
def clamp_jaw():
    part = NULLOBJ()
    return make_part("clamp_jaw", part)

@bom_part("Clamp Screw")
def clamp_screw():
    part = NULLOBJ()
    return make_part("clamp_screw", part)      

@bom_part("Fan Control Box")
def fanCTL(rotation=v(0, 0, 0), offset=v(0, 0, 0)):
    part = translate(offset) (
               rotate(rotation) (
                   cube(fan + v(wall_size, 0, wall_size), center=False)
               )
            )
    return make_part("fanctl", part)

def bedCTL(rotation=v(0, 0, 0), offset=v(0, 0, 0)):
    outer = cube(bed)
    inner = XYZ(wall_size, 2*wall_size, wall_size)(
                cube(bed - v(2*wall_size, 0, 2*wall_size)))
    part = translate(offset) (
               rotate(rotation)(
                   outer - inner
               )
            )
    return make_part("bedctl", part)

@bom_part("Base and Hook")
def base_and_hook():
    part = NULLOBJ()
    return make_part("base_and_hook", part)

def make_all_parts():
    base_and_hook()
    fanCTL()
    bedCTL()
    clamp_screw()
    clamp_jaw()


In [13]:
keep_scad=True; make_all_parts(); keep_scad=False;

True
Keeping scad files in scad
stlname == stl/base_and_hook.stl
scadname == scad/base_and_hook.scad
Keeping scad files in scad
stlname == stl/fanctl.stl
scadname == scad/fanctl.scad
Keeping scad files in scad
stlname == stl/bedctl.stl
scadname == scad/bedctl.scad
Keeping scad files in scad
stlname == stl/clamp_screw.stl
scadname == scad/clamp_screw.scad
Keeping scad files in scad
stlname == stl/clamp_jaw.stl
scadname == scad/clamp_jaw.scad
